In [ ]:
import cv2
import numpy as np
from utils.opv import OpvModel
import matplotlib.pyplot as plt
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mymodel = OpvModel("person-detection-retail-0013", device="CPU", fp="FP32", debug=True)
def DrawBoundingBoxes(predictions, image):
    canvas = image.copy()
    predictions = predictions[0][0]
    conf = 0.20
    confidence = predictions[:,2]
    topresults = predictions[(confidence>conf)]
    (h,w) = canvas.shape[:2]
    for detection in topresults:
        box = detection[3:7] * np.array([w, h, w, h])
        (xmin, ymin, xmax, ymax) = box.astype("int")
#         print(xmin)
#         print(ymin)
#         print(xmax)
#         print(ymax)
#         print(canvas.shape)
#         canvas[[[xmin:xmax,yin:ymax,]]]
#         person = canvas[[[xmin:ymin],[xmax:ymax],:]]
        person1  = canvas[xmin:xmax,ymin:ymax,:]
        print(person1)
        black = np.zeros((640,480,3),np.uint8)
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            while True:
                image = cv2.cvtColor(person1, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = pose.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                try:
                    landmarks = results.pose_landmarks.landmark
                except:
                    pass
                mp_drawing.draw_landmarks(black, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                          mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                          )
                cv2.imshow('Animated_Character', image)
                cv2.imshow('Animated Pose',black)
                black = np.zeros((480,640,3),np.uint8)
                black.fill(255)
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0,0,255), 4)
        cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0),2)
    cv2.putText(canvas, str(len(topresults))+"persons(s) detected", (50, 50),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0),2)
    
    return canvas
camera = cv2.VideoCapture(0)
while(True):
    ret,frame = camera.read()
    predictions = mymodel.Predict(frame)
    cv2.imshow("Body Gesture", DrawBoundingBoxes(predictions, frame))
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break
camera.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
from utils.opv import OpvModel
import matplotlib.pyplot as plt
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mymodel = OpvModel("person-detection-retail-0013", device="CPU", fp="FP32", debug=True)
camera = cv2.VideoCapture(0)
while True:
    ret,frame = camera.read()
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        predictions = mymodel.Predict(frame)
        canvas = frame.copy()
        predictions = predictions[0][0]
        conf = 0.20
        confidence = predictions[:,2]
        topresults = predictions[(confidence>conf)]
        (h,w) = canvas.shape[:2]
        for detection in topresults:
            box = detection[3:7] * np.array([w, h, w, h])
            (xmin, ymin, xmax, ymax) = box.astype("int")
            person1  = canvas[xmin:xmax,ymin:ymax,:]
        while True:
            image = cv2.cvtColor(person1, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=2, circle_radius=2)
                                      )
            cv2.imshow('Animated_Character', image)
            if cv2.waitKey(1) & 0xFF == ord(' '):
                break
        camera.release()
        cv2.destroyAllWindows()
#                 print(image.shape)
#                 plt.imshow(image)

#                 cv2.imshow('Animated Pose',black)
#                 black = np.zeros((480,640,3),np.uint8)
#                 black.fill(255)

#         cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0,0,255), 4)
#         cv2.putText(canvas, str(round(detection[2]*100,1))+"%", (xmin, ymin),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0),2)
#     cv2.putText(canvas, str(len(topresults))+"persons(s) detected", (50, 50),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0),2)
    
    cv2.imshow("Body Gesture", canvas)
